In [ ]:
import itertools
from typing import Iterable

from qubecalib.instrument.quel.quel1.tool.skew import Skew
from qubex.experiment import Experiment


def mux(mux_id: id):
    return [f"Q{i:02d}" for i in range(4 * mux_id, 4 * mux_id + 4)]


def flatten(x: Iterable) -> list[str]:
    return list(itertools.chain.from_iterable(x))


# mux を楽に設定するためのコード

qubits = (
    []
    # + flatten([mux(i) for i in range(0, 3)]))
    # + flatten([mux(i) for i in range(4, 8)]))
    # + flatten([mux(i) for i in range(8, 11)]))
    + flatten([mux(i) for i in [12, 13, 14]])
)

ex = Experiment(
    chip_id="64Q",
    qubits=qubits,
    config_dir="/home/shared/config",
)
qc = ex.tool.get_qubecalib()

# =============================================================================
# 以下は、qubexのsysdbに定義されていないポートやチャンネルを定義するためのコード
# qubex の yaml に追加してもらうのが望ましい
# =============================================================================

qc.sysdb.define_port(
    port_name="R23A.PORT6",
    box_name="R23A",
    port_number=6,
)
qc.sysdb.define_channel(
    channel_name="R23A.PORT6.CH0",
    port_name="R23A.PORT6",
    channel_number=0,
    ndelay_or_nwait=0,
)
qc.sysdb._relation_channel_target.append(("R23A.PORT6.CH0", "R23A.PORT6.CH0"))

qc.sysdb.define_port(
    port_name="Q73A.MON0.IN",
    box_name="Q73A",
    port_number=5,
)
qc.sysdb.define_channel(
    channel_name="Q73A.MON0.IN0",
    port_name="Q73A.MON0.IN",
    channel_number=0,
    ndelay_or_nwait=7,
)
qc.sysdb._relation_channel_target.append(("Q73A.MON0.IN0", "MON0"))

qc.sysdb.define_port(
    port_name="Q73A.MON1.IN",
    box_name="Q73A",
    port_number=12,
)
qc.sysdb.define_channel(
    channel_name="Q73A.MON1.IN0",
    port_name="Q73A.MON1.IN",
    channel_number=0,
    ndelay_or_nwait=7,
)
qc.sysdb._relation_channel_target.append(("Q73A.MON1.IN0", "MON1"))

qc.sysdb.define_port(
    port_name="Q73A.PUMP0",
    box_name="Q73A",
    port_number=3,
)
qc.sysdb.define_channel(
    channel_name="Q73A.PUMP0.CH0",
    port_name="Q73A.PUMP0",
    channel_number=0,
    ndelay_or_nwait=0,
)
qc.sysdb._relation_channel_target.append(("Q73A.PUMP0.CH0", "PUMP0"))

qc.sysdb.define_port(
    port_name="Q73A.PUMP1",
    box_name="Q73A",
    port_number=10,
)
qc.sysdb.define_channel(
    channel_name="Q73A.PUMP1.CH0",
    port_name="Q73A.PUMP1",
    channel_number=0,
    ndelay_or_nwait=0,
)
qc.sysdb._relation_channel_target.append(("Q73A.PUMP1.CH0", "PUMP1"))


# スキュー調整

In [ ]:
# まず skew 対象となる box を定義する
# 以下のコードだと qubex で確保した box が skew 対象となる
box_names = list(ex.boxes)
system = qc.create_quel1system(box_names)  # 複数の box を保持するオブジェクト

# skew のコードは Skew オブジェクトにまとめた
skew = Skew(system, qubecalib=qc)
skew.load("skew.yaml")  # load した setting は qubecalib.sysdb に適用される
qc.resync(*box_names)  # 念のなめ resync しておく

# まずは yaml ファイルに記述された skew の設定で skew を観測する
# ひとつ目の山は神 box から出した基準パルス
# ふたつ目の山は target_box が送ったパルス
# 二つ目の山のピークが縦に揃っていることが理想
skew.measure()
skew.plot()

# チェックだけならここまでで良い

# 自動で skew を調整する
# まだ動作が怪しい
skew_results = skew.adjust()
skew.plot()
print(skew_results)

# skew の調整結果を確認する
skew.measure()
skew.plot()

# 前の情報を保管するために名前を変えておいた
skew.save("skew_work.yaml")